In [1]:
import torch
import torch.nn as nn
from torchvision.models import MobileNet_V2_Weights
from torchvision import transforms, datasets
from torch.utils.data import DataLoader


In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
preprocess = MobileNet_V2_Weights.IMAGENET1K_V1.transforms()

test_dataset = datasets.ImageNet(root='../data/imagenet',
                                 split='val',
                                transform=preprocess)

In [9]:
import os
from torch.utils.data import Dataset
from PIL import Image
import json
syn_to_class = {}
with open(os.path.join("../data/imagenet", "imagenet_class_index.json"), "rb") as f:
    json_file = json.load(f)
    for class_id, v in json_file.items():
        syn_to_class[v[0]] = class_id
                
def get_class_name(entry):        
    target = syn_to_class[int(entry)]
    return target
        
class ImageNetKaggle(Dataset):
    def __init__(self, root, transform=None):
        with open(os.path.join("../data/imagenet", "imagenet_class_index.json"), "rb") as f:
            json_file = json.load(f)
            for class_id, v in json_file.items():
                syn_to_class[v[0]] = class_id
                
        self.samples = []
        self.targets = []
        self.transform = transform
        samples_dir = os.path.join(root, "val")
        for entry in os.listdir(samples_dir):
                sample_path = os.path.join(samples_dir, entry)
                for file in os.listdir(sample_path):                    
                    self.samples.append(os.path.join(sample_path, file))
                    self.targets.append(int(syn_to_class[entry]))
                
    def __len__(self):
            return len(self.samples)
        
    def __getitem__(self, idx):
            x = Image.open(self.samples[idx]).convert("RGB")
            if self.transform:
                x = self.transform(x)
            return x, self.targets[idx]

In [10]:
syn_to_class

{'n01440764': '0',
 'n01443537': '1',
 'n01484850': '2',
 'n01491361': '3',
 'n01494475': '4',
 'n01496331': '5',
 'n01498041': '6',
 'n01514668': '7',
 'n01514859': '8',
 'n01518878': '9',
 'n01530575': '10',
 'n01531178': '11',
 'n01532829': '12',
 'n01534433': '13',
 'n01537544': '14',
 'n01558993': '15',
 'n01560419': '16',
 'n01580077': '17',
 'n01582220': '18',
 'n01592084': '19',
 'n01601694': '20',
 'n01608432': '21',
 'n01614925': '22',
 'n01616318': '23',
 'n01622779': '24',
 'n01629819': '25',
 'n01630670': '26',
 'n01631663': '27',
 'n01632458': '28',
 'n01632777': '29',
 'n01641577': '30',
 'n01644373': '31',
 'n01644900': '32',
 'n01664065': '33',
 'n01665541': '34',
 'n01667114': '35',
 'n01667778': '36',
 'n01669191': '37',
 'n01675722': '38',
 'n01677366': '39',
 'n01682714': '40',
 'n01685808': '41',
 'n01687978': '42',
 'n01688243': '43',
 'n01689811': '44',
 'n01692333': '45',
 'n01693334': '46',
 'n01694178': '47',
 'n01695060': '48',
 'n01697457': '49',
 'n0169864

In [14]:
from torch.utils.data import DataLoader
from torchvision import transforms
import torch
import torchvision
from tqdm import tqdm

mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
val_transform = transforms.Compose(
            [
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize(mean, std),
            ]
        )
dataset = ImageNetKaggle("../data/imagenet", val_transform)
dataloader = DataLoader(
            dataset,
            batch_size=64, # may need to reduce this depending on your GPU 
            shuffle=False,
        )

In [15]:
def evaluate(model, device, test_loader):
    model.eval()
    
    losses = 0.0
    total_predictions = 0
    true_predictions_top1 = 0
    true_predictions_top5 = 0
    criterion = torch.nn.CrossEntropyLoss()
    
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            
            # Compute loss
            loss = criterion(outputs, targets) / inputs.size(0)
            losses += loss.item()
            
            # Top-1 predictions
            _, predicted_top1 = torch.max(outputs, 1)
            batch_true_predictions_top1 = (predicted_top1 == targets).sum().item()
            true_predictions_top1 += batch_true_predictions_top1
            
            # Top-5 predictions
            _, predicted_top5 = torch.topk(outputs, 5, dim=1)
            batch_true_predictions_top5 = sum(
                [targets[i].item() in predicted_top5[i].tolist() for i in range(targets.size(0))]
            )
            true_predictions_top5 += batch_true_predictions_top5
            
            # Update total predictions
            batch_total_predictions = outputs.size(0)
            total_predictions += batch_total_predictions
            
            # Print batch metrics
            # print(
            #     f'Batch {batch_idx}, Loss: {loss:.4f}, '
            #     f'Accuracy@1: {batch_true_predictions_top1/batch_total_predictions*100:.2f}%, '
            #     f'Accuracy@5: {batch_true_predictions_top5/batch_total_predictions*100:.2f}%'
            # )
    
    # Compute overall accuracies
    accuracy_top1 = true_predictions_top1 / total_predictions
    accuracy_top5 = true_predictions_top5 / total_predictions
    
    return accuracy_top1, accuracy_top5, losses


In [16]:
model = torchvision.models.mobilenet_v2(weights='MobileNet_V2_Weights.IMAGENET1K_V1')
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

In [17]:
model.eval().to(DEVICE)
accuracy_top1, accuracy_top5, losses = evaluate(model, DEVICE, dataloader)
print(f"acc@1: {accuracy_top1*100}%, acc@5: {accuracy_top5*100}%, loss: {losses}")

acc@1: 71.87%, acc@5: 90.31%, loss: 14.188950040435884
